In [ ]:
!pip install transformers datasets accelerate==0.20.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 67.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.5/227.5 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 59.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 13.4 MB/s eta 0:00:00


In [ ]:
import datasets
pair_ds = datasets.load_dataset('liyongsea/un_linebreak-1000')

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/668128 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/84617 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/liyongsea___parquet/liyongsea--un_linebreak-1000-bb158f3f17c3a309/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
for sample in pair_ds['train'].select(range(1000, 1200)):
    print(sample["label"], repr(sample["text"]))

False 'coverage; there might be issues that were considered\nvery important, but if theywere already receiving'
False 'very important, but if theywere already receiving\nadequate coverage, they would not make in onto the'
False 'adequate coverage, they would not make in onto the\nTen Stories list. Second, it had to consider whether the'
False 'Ten Stories list. Second, it had to consider whether the\nstories would attract the interest of the media; the'
False 'stories would attract the interest of the media; the\nDepartment could always explain why something was'
False 'Department could always explain why something was\nimportant for the world to know, but if it failed to'
False 'important for the world to know, but if it failed to\nconvince a journalist that the story was worth writing'
False 'convince a journalist that the story was worth writing\nor broadcasting, then it was wasting its time.'
True 'or broadcasting, then it was wasting its time.\n40. On the second question, relating

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer
from sklearn.model_selection import train_test_split

In [ ]:
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.float)
        }

In [ ]:
pair_ds

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 668128
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 84617
    })
})

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')


In [ ]:
small_train = pair_ds["train"]
small_test = pair_ds["test"].select(range(5000))
train_dataset = TextDataset(small_train["text"], small_train["label"], tokenizer, max_length=128)
test_dataset = TextDataset(small_test["text"], small_test["label"], tokenizer, max_length=128)

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
class TextClassifier(nn.Module):
    def __init__(self, hidden_size, num_layers, bidirectional, dropout):
        super(TextClassifier, self).__init__()
        self.embedding = nn.Embedding(tokenizer.vocab_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, num_layers=num_layers, bidirectional=bidirectional, batch_first=True)
        self.dropout = nn.Dropout(dropout)
        if bidirectional:
            self.fc = nn.Linear(2 * hidden_size, 1)
        else:
            self.fc = nn.Linear(hidden_size, 1)

    def forward(self, input_ids):
        embedded = self.embedding(input_ids)
        _, (hidden_state, _) = self.lstm(embedded)

        if self.lstm.bidirectional:
            # Concat the final forward (hidden_state[-2,:,:]) and backward (hidden_state[-1,:,:]) hidden layers
            # and apply dropout
            hidden = self.dropout(torch.cat((hidden_state[-2,:,:], hidden_state[-1,:,:]), dim = 1))
        else:
            hidden = self.dropout(hidden_state[-1,:,:])

        output = self.fc(hidden)
        return output.squeeze()


In [ ]:
def init_weights(m):
    if type(m) == nn.LSTM:
        for name, param in m.named_parameters():
            if 'weight_ih' in name:
                nn.init.xavier_uniform_(param.data)
            elif 'weight_hh' in name:
                nn.init.xavier_uniform_(param.data)
            elif 'bias' in name:
                param.data.fill_(0)


In [ ]:
# Initialize the model and send it to device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = TextClassifier
def init_weights(m):
    if type(m) == nn.LSTM:
        for name, param in m.named_parameters():
            if 'weight_ih' in name:
                nn.init.xavier_uniform_(param.data)
            elif 'weight_hh' in name:
                nn.init.xavier_uniform_(param.data)
            elif 'bias' in name:
                param.data.fill_(0)
    elif type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0)
    elif type(m) == nn.Embedding:
        nn.init.xavier_uniform_(m.weight)

model = TextClassifier(hidden_size=256, num_layers=1, bidirectional=True, dropout=0.5).to(device)
model.apply(init_weights)

TextClassifier(
  (embedding): Embedding(28996, 256)
  (lstm): LSTM(256, 256, batch_first=True, bidirectional=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=512, out_features=1, bias=True)
)

In [ ]:
# Loss function and optimizer
loss_fn = nn.BCEWithLogitsLoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
# optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
# optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)
optimizer = torch.optim.RMSprop(model.parameters(), lr=1e-3, weight_decay=1e-5)

In [ ]:
# Training loop
num_epochs = 10
step = 0
for epoch in range(num_epochs):
    model.train()
    for batch in train_dataloader:
        input_ids = batch['input_ids'].to(device)
        labels = batch['label'].to(device)

        outputs = model(input_ids)
        loss = loss_fn(outputs, labels.view(-1))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # Print loss every 10 steps
        if step % 100 == 0:
            print(f'Epoch: {epoch+1}, Step: {step}, Training Loss: {loss.item()}')

        step += 1


    # Validation at the end of the epoch
    model.eval()
    validation_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in test_dataloader:
            input_ids = batch['input_ids'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids)
            predicted = torch.round(torch.sigmoid(outputs))  # Sigmoid and rounding to get binary predictions
            total += labels.size(0)
            correct += (predicted == labels.view(-1)).sum().item()

            # Calculate validation loss
            loss = loss_fn(outputs, labels.view(-1))
            validation_loss += loss.item()

    avg_val_loss = validation_loss / len(test_dataloader)
    print(f'Validation accuracy after epoch {epoch + 1}: {correct / total:.2f}')
    print(f'Average validation loss after epoch {epoch + 1}: {avg_val_loss:.2f}')

Epoch: 1, Step: 0, Training Loss: 0.6920273900032043
Epoch: 1, Step: 100, Training Loss: 0.35656267404556274
Epoch: 1, Step: 200, Training Loss: 0.7035908102989197
Epoch: 1, Step: 300, Training Loss: 0.5348150730133057
Epoch: 1, Step: 400, Training Loss: 0.29921984672546387
Epoch: 1, Step: 500, Training Loss: 0.3837110996246338
Epoch: 1, Step: 600, Training Loss: 0.337597519159317
Epoch: 1, Step: 700, Training Loss: 0.458394318819046
Epoch: 1, Step: 800, Training Loss: 0.2568318545818329
Epoch: 1, Step: 900, Training Loss: 0.3543148934841156
Epoch: 1, Step: 1000, Training Loss: 0.46323853731155396
Epoch: 1, Step: 1100, Training Loss: 0.24681636691093445
Epoch: 1, Step: 1200, Training Loss: 0.37191885709762573
Epoch: 1, Step: 1300, Training Loss: 0.5609386563301086
Epoch: 1, Step: 1400, Training Loss: 0.3179887533187866
Epoch: 1, Step: 1500, Training Loss: 0.3344634473323822


KeyboardInterrupt: ignored